In [ ]:
pip install simcse transformers==4.2.1 scipy==1.5.4 datasets==1.2.1 pandas==1.1.5 scikit-learn==0.24.0 prettytable==2.1.0 gradio torch setuptools==49.3.0

     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 159 kB 61.7 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 979 kB 81.1 MB/s 
     |████████████████████████████████| 790 kB 79.1 MB/s 
     |████████████████████████████████| 895 kB 88.7 MB/s 
     |████████████████████████████████| 2.9 MB 60.4 MB/s 
     |████████████████████████████████| 243 kB 78.1 MB/s 
     |████████████████████████████████| 69 kB 8.0 MB/s 
     |████████████████████████████████| 206 kB 83.9 MB/s 
     |████████████████████████████████| 1.9 MB 74.4 MB/s 
     |████████████████████████████████| 961 kB 38.0 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 74.2 MB/s 
  Created wheel for simcse: filename=simcse-0.4-py3-none-any.whl size=15032 sha256=7c5592174912e07168b423d6f776d11ad88b01c2dc12f5cdd531

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
from datasets import load_metric
from simcse import SimCSE
import torch

torch.cuda.is_available()
dtype = torch.cuda.FloatTensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_folder = 'drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/'
train_path = data_folder+"train_triplets.csv"

# Exploring

In [ ]:
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")
## is supervised and case insensitive

In [ ]:
print(model)

In [ ]:
embeddings = model.encode("A WOMAN IS READING.")
embeddings1 = model.encode("A Woman Is Reading.")
embeddings2 = model.encode("a woman is reading.")

100%|██████████| 1/1 [00:00<00:00, 112.67it/s]


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

tokens = tokenizer.tokenize("What <extended> happened!")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

# ['hello', 'tensor', '##flow', '!']
# [7592, 23435, 12314, 999]

['what', '<', 'extended', '>', 'happened', '!']
[2054, 1026, 3668, 1028, 3047, 999]


In [ ]:
model.similarity(['whaaaaaaaaatttttt'], ['whaaaattttttttttt'])

100%|██████████| 1/1 [00:00<00:00, 54.18it/s]


array([[0.9895264]], dtype=float32)

In [ ]:
model = SimCSE("princeton-nlp/sup-simcse-roberta-large")
embeddings = model.encode("A WOMAN IS READING.")
embeddings1 = model.encode("A Woman Is Reading.")
embeddings2 = model.encode("a woman is reading.")

100%|██████████| 1/1 [00:00<00:00, 66.51it/s]


In [ ]:
embeddings == embeddings1

tensor([False, False, False,  ..., False, False, False])

In [ ]:
print(embeddings.size())

torch.Size([768])


In [ ]:
#Compute the cosine similarities between two groups of sentences
sentences_a = ['A woman is reading.', 'A man is playing a guitar.']
sentences_b = ['He plays guitar.', 'A woman is making a photo.']
similarities = model.similarity(sentences_a, sentences_b)
print("\nSim1:",similarities)
sentences_a = ['A woman is reading.', 'A man is playing a guitar.']
sentences_b = ['He is reading.', 'A woman is making a photo.']
similarities = model.similarity(sentences_a, sentences_b)
print("\nSim2", similarities)
sentences_a = ['A woman is reading.',  'A man is playing a guitar.']
sentences_b = ['He is reading.']
similarities = model.similarity(sentences_a, sentences_b)
print("\nSim3:",similarities)
sentences_a = ['A woman is reading.']
sentences_b = ['He is reading.']
similarities = model.similarity(sentences_a, sentences_b)
print("\nSim4:",similarities)

100%|██████████| 1/1 [00:00<00:00, 73.15it/s]


Sim1: [[0.01262089 0.3446951 ]
 [0.8938425  0.04842839]]

Sim2 [[0.65000594 0.3446951 ]
 [0.264742   0.04842839]]

Sim3: [[0.65000594]
 [0.26474205]]

Sim4: [[0.6500059]]


In [ ]:
#build index for a group of sentences and search among them
sentences = ['what_extended','what']
model.build_index(sentences)
results = model.search("whaaaaaaaaaaat")
print("\nRes1:",results)

11/27/2021 21:44:56 - WARNING - simcse.tool -   Fail to import faiss. If you want to use faiss, install faiss through PyPI. Now the program continues with brute force search.
11/27/2021 21:44:56 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 1/1 [00:00<00:00, 57.24it/s]
11/27/2021 21:44:56 - INFO - simcse.tool -   Building index...
11/27/2021 21:44:56 - INFO - simcse.tool -   Finished
100%|██████████| 1/1 [00:00<00:00, 65.92it/s]


Res1: [('what', 1.0000001192092896)]


In [ ]:
#build index for a group of sentences and search among them
sentences = ['She has 10 apples','She has 100000 apples']
model.build_index(sentences)
results = model.search("He has 100000 apples")
print("\nRes1:",results)
results = model.search("He has 10 apples.")
print("\nRes2:",results)

11/26/2021 00:52:26 - WARNING - simcse.tool -   Fail to import faiss. If you want to use faiss, install faiss through PyPI. Now the program continues with brute force search.
11/26/2021 00:52:26 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 1/1 [00:00<00:00, 50.01it/s]
11/26/2021 00:52:26 - INFO - simcse.tool -   Building index...
11/26/2021 00:52:26 - INFO - simcse.tool -   Finished
100%|██████████| 1/1 [00:00<00:00, 73.45it/s]


Res1: [('She has 100000 apples', 0.8586735725402832), ('She has 10 apples', 0.6768637299537659)]

Res2: [('She has 10 apples', 0.8687605857849121), ('She has 100000 apples', 0.6759915947914124)]


In [ ]:
#build index for a group of sentences and search among them
sentences = ['A woman is reading.', 'A man is playing a guitar.']
model.build_index(sentences)
results = model.search("He plays guitar.")
print("\nRes1:",results)
results = model.search("She plays guitar.")
print("\nRes2:",results)

11/26/2021 00:52:29 - WARNING - simcse.tool -   Fail to import faiss. If you want to use faiss, install faiss through PyPI. Now the program continues with brute force search.
11/26/2021 00:52:29 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 1/1 [00:00<00:00,  5.57it/s]
11/26/2021 00:52:29 - INFO - simcse.tool -   Building index...
11/26/2021 00:52:29 - INFO - simcse.tool -   Finished
100%|██████████| 1/1 [00:00<00:00, 70.97it/s]


Res1: [('A man is playing a guitar.', 0.8938425779342651)]

Res2: [('A man is playing a guitar.', 0.6488244533538818)]


In [ ]:
#build index for a group of sentences and search among them
sentences = ["whaaat!", "whaaattttt"]
model.build_index(sentences)
results = model.search("what")
print("\nRes1:",results)
sentences_a = ["whaat", "whatt"]
sentences_b = ["what"]
similarities = model.similarity(sentences_a, sentences_b)
print("\nSim1:",similarities)

11/26/2021 00:57:03 - WARNING - simcse.tool -   Fail to import faiss. If you want to use faiss, install faiss through PyPI. Now the program continues with brute force search.
11/26/2021 00:57:03 - INFO - simcse.tool -   Encoding embeddings for sentences...
100%|██████████| 1/1 [00:00<00:00, 53.17it/s]
11/26/2021 00:57:03 - INFO - simcse.tool -   Building index...
11/26/2021 00:57:03 - INFO - simcse.tool -   Finished
100%|██████████| 1/1 [00:00<00:00, 63.04it/s]


Res1: []

Sim1: [[0.6356628]
 [0.874833 ]]


In [ ]:
# Uncomment the code below if we need to get the SimCSE code from their git
# !wget https://github.com/princeton-nlp/SimCSE/archive/refs/heads/main.zip
# !unzip main.zip
# !cd SimCSE-main/SentEval/data/downstream/ && wget https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/senteval.tar && tar xvf senteval.tar
# !ls SimCSE-main/SentEval/data/downstream/STS/STSBenchmark/
# !wget https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/nli_for_simcse.csv

# Train

In [ ]:
#Run this!

%cd ../content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets
!pwd

/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets
/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets


## Deleting a folder - USE WITH CAUTION!


In [ ]:
# import shutil
# shutil.rmtree('/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/twiBot-similarity-20epochs-sup-simcse-bert-base-uncased')

## Training with TwiBot - GPU

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data


In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../train_triplets.csv \
    --output_dir ../twiBot-sup-simcse-bert-base-uncased \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/19/2021 23:21:27 - INFO - __main__ -   PyTorch: setting up devices
11/19/2021 23:21:27 - INFO - __main__ -   Set device to CUDA
11/19/2021 23:21:27 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/19/2021 23:21:27 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Nov1

## Training with TwiBot 20 epochs - GPU

In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../train_triplets.csv \
    --output_dir ../twiBot-20epochs-sup-simcse-bert-base-uncased \
    --num_train_epochs 20 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --pooler_type cls \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/18/2021 11:35:58 - INFO - __main__ -   PyTorch: setting up devices
11/18/2021 11:35:58 - INFO - __main__ -   Set device to CUDA
11/18/2021 11:35:58 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/18/2021 11:35:58 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-20epochs-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=20.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir=

## Training with TwiBot dataset built on *similarity* - GPU

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data


In [ ]:
# Don't run again!

# Extract only the tweet data from the dataset and copy to train_triplets_with_similarity_only_tweets.csv
df = pd.read_csv("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/train_triplets_with_similarity.csv")
df_only_tweets = df[['sent0', 'sent1', 'hard_neg']]
df_only_tweets.to_csv("train_triplets_with_similarity_only_tweets.csv", index=False)

In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../train_triplets_with_similarity_only_tweets.csv \
    --output_dir ../twiBot-similarity-sup-simcse-bert-base-uncased \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/23/2021 03:24:35 - INFO - __main__ -   PyTorch: setting up devices
11/23/2021 03:24:35 - INFO - __main__ -   Set device to CUDA
11/23/2021 03:24:35 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/23/2021 03:24:35 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-similarity-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir

In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../train_triplets_with_similarity_only_tweets.csv \
    --output_dir ../twiBot-similarity-10epochs-sup-simcse-bert-base-uncased \
    --num_train_epochs 10 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/23/2021 04:13:50 - INFO - __main__ -   PyTorch: setting up devices
11/23/2021 04:13:50 - INFO - __main__ -   Set device to CUDA
11/23/2021 04:13:50 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/23/2021 04:13:50 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-similarity-10epochs-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, l

## Training with TwiBot dataset built on *similarity* - 15 epochs - GPU

In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../train_triplets_with_similarity_only_tweets.csv \
    --output_dir ../twiBot-similarity-15epochs-sup-simcse-bert-base-uncased \
    --num_train_epochs 15 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/24/2021 00:26:15 - INFO - __main__ -   PyTorch: setting up devices
11/24/2021 00:26:15 - INFO - __main__ -   Set device to CUDA
11/24/2021 00:26:15 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/24/2021 00:26:15 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-similarity-15epochs-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=15.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, l

## Training with TwiBot dataset built on *similarity* - 20 epochs - GPU

In [ ]:
!cd SimCSE-main && python train.py --model_name_or_path ../twiBot-similarity-10epochs-sup-simcse-bert-base-uncased \
    --train_file ../train_triplets_with_similarity_only_tweets.csv \
    --output_dir ../twiBot-similarity-20epochs-sup-simcse-bert-base-uncased \
    --num_train_epochs 20 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/23/2021 11:29:08 - INFO - __main__ -   PyTorch: setting up devices
11/23/2021 11:29:08 - INFO - __main__ -   Set device to CUDA
11/23/2021 11:29:08 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/23/2021 11:29:08 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../twiBot-similarity-20epochs-sup-simcse-bert-base-uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=20.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, l

## Training with 360k TwiBot dataset built on *similarity* with metadata - 1 epoch

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/Model_TwiBot_Similarity_With_Metadata_360k_Tweets


In [ ]:
!cd ../SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_for_simcse.csv \
    --output_dir ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 48 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/27/2021 14:26:21 - INFO - __main__ -   PyTorch: setting up devices
11/27/2021 14:26:21 - INFO - __main__ -   Set device to CUDA
11/27/2021 14:26:21 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/27/2021 14:26:21 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINE

## Training with 360k TwiBot dataset built on similarity with metadata - no tags for uppercase - 1 epoch LR 5e-5
--model_name_or_path bert-base-uncased

In [ ]:
!cd ../SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_for_simcse.csv \
    --output_dir ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_bert_base_uncased \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 48 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/28/2021 08:06:07 - INFO - __main__ -   PyTorch: setting up devices
11/28/2021 08:06:07 - INFO - __main__ -   Set device to CUDA
11/28/2021 08:06:07 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/28/2021 08:06:07 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_bert_base_uncased', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler

## Training with 360k TwiBot dataset built on similarity with metadata - no tags for uppercase - 1 epoch LR 5e-5
-model_name_or_path princeton-nlp/sup-simcse-roberta-base

In [ ]:
!cd ../SimCSE-main && python train.py --model_name_or_path princeton-nlp/sup-simcse-roberta-base \
    --train_file ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_for_simcse.csv \
    --output_dir ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR5e-5_roberta_base \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 5e-5 \
    --max_seq_length 48 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

## Training with 360k TwiBot dataset built on similarity with metadata - no tags for uppercase - 1 epoch LR 1e-5
--model_name_or_path bert-base-uncased

In [ ]:
!cd ../SimCSE-main && python train.py --model_name_or_path bert-base-uncased \
    --train_file ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_for_simcse.csv \
    --output_dir ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR1e-5_bert_base_uncased \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 1e-5 \
    --max_seq_length 48 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

## Training with 360k TwiBot dataset built on similarity with metadata - no tags for uppercase - 1 epoch LR 1e-5
-model_name_or_path princeton-nlp/sup-simcse-roberta-base

In [ ]:
!cd ../SimCSE-main && python train.py --model_name_or_path princeton-nlp/sup-simcse-roberta-base \
    --train_file ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/train_all_data_triplets_with_similarity_for_simcse.csv \
    --output_dir ../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR1e-5_roberta_base \
    --num_train_epochs 1 \
    --per_device_train_batch_size 32 \
    --learning_rate 1e-5 \
    --max_seq_length 48 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \

11/30/2021 01:40:36 - INFO - __main__ -   PyTorch: setting up devices
11/30/2021 01:40:36 - INFO - __main__ -   Set device to CUDA
11/30/2021 01:40:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1 distributed training: False, 16-bits training: False
11/30/2021 01:40:36 - INFO - __main__ -   Training/evaluation parameters OurTrainingArguments(output_dir='../Model_TwiBot_Similarity_With_Metadata_360k_Tweets/twiBot-360k-similarity-metadata-1epoch_LR1e-5_roberta_base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type

# Load model

In [ ]:
ls drive/MyDrive/twiBot-20epochs-sup-simcse-bert-base-uncased

ls: cannot access 'drive/MyDrive/twiBot-20epochs-sup-simcse-bert-base-uncased': No such file or directory


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1ygEmNCP-_htwwx_sBJE2NdJ3Nb7aJhr3/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data


In [ ]:
sent1 = ['the butlers visitor experience will leave you with cherished memories and your tastebuds longing for more butlers chocolate . check availability here <_url_> <_url_>']
sent2 = ['angel food cake hack to satisfy your cravings <_url_>']
sen_neg = ['adding seasonal words to search terms can help your items surface in search results . <_url_> ( seller central login required )']

In [ ]:
# Trained on the twibot-similarity-based dataset - 20 epoch
model_twi = model = SimCSE("/content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/twiBot-similarity-20epochs-sup-simcse-bert-base-uncased")
# model_twi

sim1= model_twi.similarity(sent1, sent2)
sim2 = model_twi.similarity(sent1, sen_neg)
print("\nPos:",sim1)
print("\nNeg:",sim2)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/CSCI_544_NLP_Project/Twitter_Bot_Detection/Data/twiBot-similarity-20epochs-sup-simcse-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1/1 [00:00<00:00, 105.35it/s]


Pos: [[0.8136509]]

Neg: [[0.5301692]]
